In [1]:
import xarray as xr
import numpy as np
import s3fs

In [44]:
def open_zarr(year, month, datestart, dateend):
    fs = s3fs.S3FileSystem(anon=True)

    datestring = 'era5-pds/zarr/{year}/{month:02d}/data/'.format(year=year, month=month)

    precip_zarr = xr.open_zarr(s3fs.S3Map(datestring + 'precipitation_amount_1hour_Accumulation.zarr/', s3=fs))
    precip_zarr = precip_zarr.sel(time1=slice(np.datetime64(datestart), np.datetime64(dateend)))

    return precip_zarr.precipitation_amount_1hour_Accumulation

In [53]:
%time precip_m1 = open_zarr(1987, 10, '1987-10-01', '1987-10-30 23:59')
%time precip_m2 = open_zarr(1987, 11, '1987-11-01', '1987-11-30 23:59')
%time precip_m3 = open_zarr(1987, 12, '1987-12-01', '1987-12-30 23:59')

CPU times: user 304 ms, sys: 15 ms, total: 319 ms
Wall time: 13.4 s
CPU times: user 288 ms, sys: 12 ms, total: 300 ms
Wall time: 12.9 s
CPU times: user 302 ms, sys: 13 ms, total: 315 ms
Wall time: 13 s


In [54]:
precip_m1

<xarray.DataArray 'precipitation_amount_1hour_Accumulation' (time1: 720, lat: 721, lon: 1440)>
dask.array<getitem, shape=(720, 721, 1440), dtype=float32, chunksize=(372, 150, 150), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * time1    (time1) datetime64[ns] 1987-10-01 ... 1987-10-30T23:00:00
Attributes:
    long_name:       Total precipitation
    nameCDM:         Total_precipitation_1hour_Accumulation
    nameECMWF:       Total precipitation
    product_type:    forecast
    shortNameECMWF:  tp
    standard_name:   precipitation_amount
    units:           m

In [56]:
%time precip = np.stack((precip_m1.values, precip_m2.values, precip_m3.values))
precip.shape

CPU times: user 19.2 s, sys: 7.65 s, total: 26.8 s
Wall time: 41.4 s


(3, 720, 721, 1440)

In [99]:
%time a = np.mean(precip, axis=(3, 2, 0))

CPU times: user 615 ms, sys: 2.44 ms, total: 618 ms
Wall time: 616 ms


In [91]:
import iarray as ia

storage = ia.Storage(chunkshape=(3, 64, 64, 128), blockshape=(3, 16, 16, 64))
ia.set_config(ia.Config())

%time ia_precip = ia.numpy2iarray(precip, storage=storage)
ia_precip
ia_precip.cratio

CPU times: user 20.6 s, sys: 1.25 s, total: 21.8 s
Wall time: 10.2 s


10.701418067171831

In [74]:
ia_precip.chunkshape, ia_precip.blockshape

((3, 64, 64, 128), (3, 16, 16, 64))

In [98]:
ia.set_config(nthreads=0)
%time b = ia.mean(ia_precip, axis=(3, 2, 0))
b

CPU times: user 8.06 s, sys: 157 ms, total: 8.22 s
Wall time: 1.02 s


<IArray (720,) np.float32>

In [93]:
%time ia.save(ia_precip, "precip-3m.iarr")

CPU times: user 61.3 ms, sys: 1.52 s, total: 1.58 s
Wall time: 1.61 s
